### Creating a stupid model with 0 feature engineering for comparison purposes

In [18]:
# imports
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
import pandas as pd
import tensorflow as tf
import keras_tuner as kt
import os

In [19]:
# read in the audi csv located within the data directory
df = pd.read_csv("../../data/car_price_prediction.csv")
df.head()

,ID,Price,Levy,Manufacturer,Model,Prod. year,Category,Leather interior,Fuel type,Engine volume,Mileage,Cylinders,Gear box type,Drive wheels,Doors,Wheel,Color,Airbags
0,45654403,13328,1399,LEXUS,RX 450,2010,Jeep,Yes,Hybrid,3.5,186005 km,6.0,Automatic,4x4,04-May,Left wheel,Silver,12
1,44731507,16621,1018,CHEVROLET,Equinox,2011,Jeep,No,Petrol,3,192000 km,6.0,Tiptronic,4x4,04-May,Left wheel,Black,8
2,45774419,8467,-,HONDA,FIT,2006,Hatchback,No,Petrol,1.3,200000 km,4.0,Variator,Front,04-May,Right-hand drive,Black,2
3,45769185,3607,862,FORD,Escape,2011,Jeep,Yes,Hybrid,2.5,168966 km,4.0,Automatic,4x4,04-May,Left wheel,White,0
4,45809263,11726,446,HONDA,FIT,2014,Hatchback,Yes,Petrol,1.3,91901 km,4.0,Automatic,Front,04-May,Left wheel,Silver,4


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19237 entries, 0 to 19236
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ID                19237 non-null  int64  
 1   Price             19237 non-null  int64  
 2   Levy              19237 non-null  object 
 3   Manufacturer      19237 non-null  object 
 4   Model             19237 non-null  object 
 5   Prod. year        19237 non-null  int64  
 6   Category          19237 non-null  object 
 7   Leather interior  19237 non-null  object 
 8   Fuel type         19237 non-null  object 
 9   Engine volume     19237 non-null  object 
 10  Mileage           19237 non-null  object 
 11  Cylinders         19237 non-null  float64
 12  Gear box type     19237 non-null  object 
 13  Drive wheels      19237 non-null  object 
 14  Doors             19237 non-null  object 
 15  Wheel             19237 non-null  object 
 16  Color             19237 non-null  object

### Unoptimized Model

In [21]:
unoptimized_df_dummy = pd.get_dummies(audi_df, columns=["Levy", "Manufacturer", "Model", "Category", "Leather interior", "Fuel type", "Engine volume", "Mileage", "Gear box type", "Drive wheels", "Doors", "Wheel", "Color"], dtype="int64")
unoptimized_df_dummy.head()

,ID,Price,Prod. year,Cylinders,Airbags,Levy_-,Levy_1011,Levy_1016,Levy_1017,Levy_1018,...,Color_Green,Color_Grey,Color_Orange,Color_Pink,Color_Purple,Color_Red,Color_Silver,Color_Sky blue,Color_White,Color_Yellow
0,45654403,13328,2010,6.0,12,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,44731507,16621,2011,6.0,8,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,45774419,8467,2006,4.0,2,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,45769185,3607,2011,4.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,45809263,11726,2014,4.0,4,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [22]:
# split preprocessed data into features and target arrays
y = unoptimized_df_dummy["Price"]
X = unoptimized_df_dummy.drop(columns=["Price"])

# split this further into testing and training sets
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [23]:
# create a scaler for X
scaler = StandardScaler()

# fit the scaler
X_scaler = scaler.fit(X_train)

# scale the X data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# reshape y
y_train = np.array(y_train).reshape(-1, 1)
y_test = np.array(y_test).reshape(-1, 1)

# create a scaler for y
y_scaler = scaler.fit(y_train)

# scale the y data
y_train_scaled = y_scaler.transform(y_train)
y_test_scaled = y_scaler.transform(y_test)

In [27]:
# define the model
nn = tf.keras.models.Sequential()

# add layers
nn.add(tf.keras.layers.Dense(units=256, activation="relu", input_dim=len(X.columns)))
nn.add(tf.keras.layers.Dense(units=128, activation="relu"))
nn.add(tf.keras.layers.Dense(units=1))

# check structure of the model
nn.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 256)            │     2,575,616 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,608,641 (9.95 MB)

 Trainable params: 2,608,641 (9.95 MB)

 Non-trainable params: 0 (0.00 B)

In [28]:
# compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam")

In [29]:
# train the model
fit_model = nn.fit(X_train_scaled, y_train_scaled, epochs=100)

Epoch 1/100
451/451 ━━━━━━━━━━━━━━━━━━━━ 12s 24ms/step - loss: -0.0347
Epoch 2/100
451/451 ━━━━━━━━━━━━━━━━━━━━ 14s 32ms/step - loss: 0.0214
Epoch 3/100
451/451 ━━━━━━━━━━━━━━━━━━━━ 12s 26ms/step - loss: -0.0421
Epoch 4/100
451/451 ━━━━━━━━━━━━━━━━━━━━ 11s 24ms/step - loss: -0.1769
Epoch 5/100
451/451 ━━━━━━━━━━━━━━━━━━━━ 11s 25ms/step - loss: 0.0047
Epoch 6/100
451/451 ━━━━━━━━━━━━━━━━━━━━ 11s 25ms/step - loss: 0.1338
Epoch 7/100
451/451 ━━━━━━━━━━━━━━━━━━━━ 11s 25ms/step - loss: -0.1384
Epoch 8/100
451/451 ━━━━━━━━━━━━━━━━━━━━ 11s 24ms/step - loss: 0.0041
Epoch 9/100
451/451 ━━━━━━━━━━━━━━━━━━━━ 11s 25ms/step - loss: -0.1488
Epoch 10/100
451/451 ━━━━━━━━━━━━━━━━━━━━ 11s 24ms/step - loss: -0.0171
Epoch 11/100
451/451 ━━━━━━━━━━━━━━━━━━━━ 11s 25ms/step - loss: -0.0095
Epoch 12/100
451/451 ━━━━━━━━━━━━━━━━━━━━ 11s 24ms/step - loss: -0.2668
Epoch 13/100
451/451 ━━━━━━━━━━━━━━━━━━━━ 11s 24ms/step - loss: -0.2010
Epoch 14/100
451/451 ━━━━━━━━━━━━━━━━━━━━ 11s 25ms/step - loss: -0.1664
Epoch

In [30]:
# make predictions using test data
predictions = nn.predict(X_test_scaled)

151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


In [31]:
# evaluate the model using test data
mean_abs_err = mean_absolute_error(y_test_scaled, predictions)
mean_sq_err = mean_squared_error(y_test_scaled, predictions)
r2 = r2_score(y_test_scaled, predictions)

print(f"Mean Absolute Error: {mean_abs_err}\nMean Squared Error: {mean_sq_err}\nR^2: {r2}")

Mean Absolute Error: 6.37314751077402
Mean Squared Error: 415.2775441315334
R^2: -0.11039485253986925


In [32]:
# predict the first test value using model
y_pred_scaled = nn.predict(X_test_scaled[0].reshape(1, -1))
y_pred_original = y_scaler.inverse_transform(y_pred_scaled)
print(f"Prediction: {y_pred_original[0][0]}")

# actual first test value
y_test_original = y_scaler.inverse_transform(y_test_scaled[0].reshape(1, -1))
print(f"Actual: {y_test_original[0][0]}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Prediction: -130160.234375
Actual: 941.0


In [ ]:
# save the model
nn.save("stupid_model.keras")

### Testing Saved Model

In [35]:
loaded_nn = tf.keras.models.load_model("stupid_model.keras")

# predict the first test value using loaded model
y_pred_scaled = loaded_nn.predict(X_test_scaled[0].reshape(1, -1))
y_pred_original = y_scaler.inverse_transform(y_pred_scaled)
print(f"Prediction: {y_pred_original[0][0]}")

# actual first test value
y_test_original = y_scaler.inverse_transform(y_test_scaled[0].reshape(1, -1))
print(f"Actual: {y_test_original[0][0]}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Prediction: -130160.234375
Actual: 941.0
